# Wellcome to **eco2ai** tutorial

Install the following libraries into your python environment, if they are not already installed:

***pip install numpy pandas matplotlib pillow pytorch***

Install the eco2ai library in your python environment:

***pip install eco2ai***

and run this jupyter notebook as a usage example!

In [ ]:
import eco2ai
from eco2ai import track

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import random
from PIL import Image
import os

# Preparing MNIST Dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform, download=True)

# Display some example images
fig, axs = plt.subplots(1, 10, figsize=(25, 3))
for i in range(10):
    label_indexes = [idx for idx, (_, label) in enumerate(train_dataset) if label == i]
    index = random.choice(label_indexes)
    img, _ = train_dataset[index]
    axs[i].imshow(img.squeeze(), cmap='gray')
plt.show()

# Data loaders
batch_size = 256
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Define CNN model
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.batch_norm = nn.BatchNorm2d(1)
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 16, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout = nn.Dropout(0.25)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(16 * 14 * 14, 32)
        self.fc2 = nn.Linear(32, 10)

    def forward(self, x):
        x = self.batch_norm(x)
        x = torch.relu(self.conv1(x))
        x = torch.relu(self.conv2(x))
        x = self.pool(x)
        x = self.dropout(x)
        x = self.flatten(x)
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# Initialize tracker
tracker = eco2ai.Tracker(project_name="mnist", experiment_description="Convolutional model")
tracker.start()

# Training and evaluation
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = SimpleCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

# Training loop
def train(model, loader, criterion, optimizer, epochs=2):
    history = {'train_acc': [], 'val_acc': []}
    for epoch in range(epochs):
        model.train()
        correct, total = 0, 0
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            outputs = model(x)
            loss = criterion(outputs, y)
            loss.backward()
            optimizer.step()
            
            _, predicted = outputs.max(1)
            total += y.size(0)
            correct += (predicted == y).sum().item()

        train_acc = correct / total
        history['train_acc'].append(train_acc)

        # Validation loop
        model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for x, y in test_loader:
                x, y = x.to(device), y.to(device)
                outputs = model(x)
                _, predicted = outputs.max(1)
                total += y.size(0)
                correct += (predicted == y).sum().item()

        val_acc = correct / total
        history['val_acc'].append(val_acc)

        print(f"Epoch {epoch + 1}/{epochs}, Train Accuracy: {train_acc:.4f}, Val Accuracy: {val_acc:.4f}")

    return history

history = train(model, train_loader, criterion, optimizer)

# Plot accuracy
plt.plot(history['train_acc'], label='Train Accuracy')
plt.plot(history['val_acc'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

tracker.stop()

# Read emission data
df = pd.read_csv('emission.csv', sep=',')
print(df)

# Using decorators for tracking
@track
def train_simple_model():
    model = nn.Sequential(
        nn.BatchNorm2d(1),
        nn.Conv2d(1, 16, kernel_size=3, padding=1),
        nn.Flatten(),
        nn.Linear(16 * 28 * 28, 10)
    ).to(device)

    optimizer = optim.Adam(model.parameters())
    history = train(model, train_loader, criterion, optimizer)
    plt.plot(history['train_acc'], label='Train Accuracy')
    plt.plot(history['val_acc'], label='Validation Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()

train_simple_model()

df = pd.read_csv('emission.csv', sep=',')
print(df)
